In [1]:
%pip install azure-search-documents --pre
%pip show azure-search-documents

StatementMeta(smallSpark, 4, 7, Finished, Available)

  Using cached azure_search_documents-11.4.0b8-py3-none-any.whl (305 kB)
  Using cached azure_core-1.29.3-py3-none-any.whl (191 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages, outside environment /nfs4/pyenv-31d5516c-2356-46ad-98d3-5cc14c963d69
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.22.1
    Not uninstalling azure-core at /home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages, outside environment /nfs4/pyenv-31d5516c-2356-46ad-98d3-5cc14c963d69
    Can't uninstall 'azure-core'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

Name: azure-search-documents
Version: 11.4.0b8
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsoft Corporation
Author-email: ascl@microsoft.com
License: MIT License
Location: /nfs4/pyenv-31d5516c-2356-46ad-98d3-5cc14c963d69/lib/python3.8/site-packages
Requires: azure-common, azure-core, isodate
Required-by: 
Note: you may need to restart the kernel to use updated packages.



In [2]:
%pip install openai fsspec

StatementMeta(, 4, -1, Finished, Available)

  Using cached openai-0.27.9-py3-none-any.whl (75 kB)
You should consider upgrading via the '/nfs4/pyenv-31d5516c-2356-46ad-98d3-5cc14c963d69/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import openai
import os
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)
import fsspec

StatementMeta(smallSpark, 4, 21, Finished, Available)

In [4]:
service_name = os.getenv("AZURE_SEARCH_SERVICE_NAME")
admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

# Create an SDK client
endpoint = "https://{}.search.windows.net/".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

StatementMeta(smallSpark, 4, 16, Finished, Available)

In [5]:
# try:
#     result = admin_client.delete_index(index_name)
#     print ('Index', index_name, 'Deleted')
# except Exception as ex:
#     print (ex)

StatementMeta(smallSpark, 4, 17, Finished, Available)

In [6]:
# Specify the index schema
name = index_name
fields = [
        SimpleField(name='Key', type=SearchFieldDataType.String, key=True),
        SimpleField(name="ResourceURL", type=SearchFieldDataType.String),
        SearchableField(name="Who", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="What", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="When", type=SearchFieldDataType.String, sortable=True),
    ]
cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles = []


StatementMeta(smallSpark, 4, 18, Finished, Available)

In [7]:
index = SearchIndex(
    name=name,
    fields=fields,
    scoring_profiles=scoring_profiles,
    cors_options=cors_options)

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

StatementMeta(smallSpark, 4, 19, Finished, Available)

(ResourceNameAlreadyInUse) Cannot create index 'ri-innovation-quickstart' because it already exists.
Code: ResourceNameAlreadyInUse
Message: Cannot create index 'ri-innovation-quickstart' because it already exists.
Exception Details:	(CannotCreateExistingIndex) Cannot create index 'ri-innovation-quickstart' because it already exists.
	Code: CannotCreateExistingIndex
	Message: Cannot create index 'ri-innovation-quickstart' because it already exists.


Read resource CSVs into a dataframe

In [ ]:
all_files = os.getenv("AZURE_SEARCH_FILES").split(",") #Provide exact path to the files

li = []

n_rows = 0

adls_account_name = os.getenv("ADLS_ACCOUNT_NAME") #Provide exact ADLS account name
adls_account_key = os.getenv("ADLS_ACCOUNT_KEY") #Provide exact ADLS account key

for filename in all_files:
    try:
        # fsspec_handle = fsspec.open(filename, account_name=adls_account_name, account_key=adls_account_key)
        # with fsspec_handle.open() as f:
        # df = pandas.read_csv(f)
        df = pd.read_csv(filename, index_col=None, header=0) if 'csv' in filename else pd.read_excel(filename, index_col=None, header=0)
        n_rows += len(df)
        li.append(df)
    except Exception as e:
        print(f"Exception: {e}")

resource_df = pd.concat(li, axis=0, ignore_index=True)

print("There are ", n_rows, " documents in the dataset.")

StatementMeta(smallSpark, 4, 25, Finished, Available)

Exception: module 'fsspec_wrapper' has no attribute 'AzureBlobFileSystem'


ValueError: No objects to concatenate

In [12]:
# dissertation_df = dissertation_df.dropna()
resource_df

StatementMeta(smallSpark, 46, 29, Finished, Available)

,link,who,what,when,Unnamed: 0
0,https://ri-business.com/the-competition/about-...,RI Business Competition,The Rhode Island Business Competition aims to ...,2023,NaN
1,https://www.branchfood.com/about-us,branchfood,At Branchfood we facilitate the connections th...,2023,NaN
2,https://masschallenge.org/mission/,Mass Challenge,"Founded in 2009 in Boston, MA, MassChallenge’s...",2023,NaN
3,https://www.sea-ahead.com/mission,Sea Ahead,"In 2018, we realized that early-stage startups...",2023,NaN
4,https://401techbridge.org/401-tech-bridge-mass...,401 Tech Bridge,401 Tech Bridge and MassChallenge partnered on...,2023,NaN
...,...,...,...,...,...
7081,https://web.uri.edu/catalog/#/courses/Hki24HDw...,University of Rhode Island College of Arts and...,Course name: WRT646 - Seminar in Writing Studi...,2022-2023,NaN
7082,https://web.uri.edu/catalog/#/courses/ryIHVBvv...,University of Rhode Island College of Arts and...,Course name: WRT647 - Seminar in Research Meth...,2022-2023,NaN
7083,https://web.uri.edu/catalog/#/courses/H1-DF4rP...,University of Rhode Island College of Arts and...,Course name: WRT691 - Independent Study in Rhe...,2022-2023,NaN
7084,https://web.uri.edu/catalog/#/courses/HyPqVSww...,University of Rhode Island College of Arts and...,Course name: WRT699 - Doctoral Dissertation Re...,2022-2023,NaN


Iterate over resources and construct a formatted object for each to add to search index

In [13]:
documents = []

for index, row in resource_df.iterrows():
    documents.append(
        {
            "@search.action": "upload",
            "Key": str(index),
            "ResourceURL": str(row.link),
            "Who": str(row.who),
            "What": str(row.what),
            "When": str(row.when)
        }
    )

StatementMeta(smallSpark, 46, 30, Finished, Available)

In [14]:
documents[:2]

StatementMeta(smallSpark, 46, 31, Finished, Available)

[{'@search.action': 'upload',
  'Key': '0',
  'ResourceURL': 'https://ri-business.com/the-competition/about-the-competition',
  'Who': 'RI Business Competition',
  'What': "The Rhode Island Business Competition aims to create and foster growth companies in Rhode IslaThe Rhode Island Business Competition aims to create and foster growth companies in Rhode Island that will increase local employment. It's all about helping emerging entrepreneurs achieve their dreams. Established in 2000, the Rhode Island Business Competition is supported by a broad range of sponsors that include private businesses, investors, foundations, colleges and universities, public entities, nonprofit organizations, and former competitors. Today, it is the leading community-supported business plan competition in the Northeast. The Competition encourages ideas for new businesses, as well as from early stage companies. An early stage, or seed stage, company typically is at the inventor stage where there is an idea, a

Upload documents to search index

In [15]:
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

StatementMeta(smallSpark, 46, 32, Finished, Available)

Upload of new document succeeded: True


In [21]:
results = search_client.search(
                        'What courses are available abour artificial intelligence?', 
                        filter=None,
                        query_language="en-us", 
                        top=10)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print('/n/n' + str(len(result['What']) / 5))
    print(result)

StatementMeta(smallSpark, 46, 38, Finished, Available)

Total Documents Matching Query: None
/n/n166.0
{'What': 'Computational Intelligence and Self-Adaptive Systems (CISA) Laboratory. Though innovative research, this lab designs intelligent computing systems that can think and act like a human brain. Visit the lab’s website.\n\n\nWelcome to the Computational Intelligence and Self-Adaptive Systems (CISA) Laboratory!\n\n\nThe Computational Intelligence and Self-Adaptive Systems (CISA) Laboratory at the Department of Electrical, Computer, and Biomedical Engineering at the University of Rhode Island (URI) focuses on the fundamental research on computational intelligence, with a particular focus on neural networks, deep learning, reinforcement learning, and data mining. Please explore our website to find out more information […]\nContinue reading "Welcome to the Computational Intelligence and Self-Adaptive Systems (CISA) Laboratory!" → \n\n\n\n', 'Key': '2690', 'When': '2023', 'Who': 'Computational Intelligence and Self-Adaptive Systems (CISA) 

Testing with GPT

In [ ]:

def is_RI_innovation_question(user_text):
    """
    NOTE: Test this empirically to determine performance?
    """
    # VERY IMPORTANT: Need to move the API key and model deployment to a secure location, e.g. Azure Key Vault.
    openai.api_type = "azure"
    openai.api_base = os.getenv("AZURE_OAI_API_BASE")
    openai.api_version = os.getenv("AZURE_OAI_API_VERSION")
    openai.api_key = os.getenv("AOAI_API_KEY") # os.environ["AZURE_OAI_API_KEY"]""

    MODEL_DEPLOYMENT = os.getenv("MODEL_DEPLOYMENT") # os.environ["AZURE_OAI_MODEL_DEPLOYMENT"]

    prompt = f"""
        Determine whether the given query is related to entrepreneurship, innovation, or business.
        Query 1: What is the meaning of life?
        Related 1:False
        Query 2: How can I build a successful technology company?
        Related 2:True
        Query 3: Who are you?
        Related 3:False
        Query 4: What resources are available for blue tech entrepreneurs in RI?
        Related 4:True
        Query 5: {user_text}
        Related 5: 
    """

    response = openai.Completion.create(
        engine=MODEL_DEPLOYMENT,
        prompt=prompt,
        temperature=0.1,
        max_tokens=2,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    response = response['choices'][0]['text'] 

    return response


def is_academic_question(user_text):

    # VERY IMPORTANT: Need to move the API key and model deployment to a secure location, e.g. Azure Key Vault.
    openai.api_type = "azure"
    openai.api_base = os.getenv("AZURE_OAI_API_BASE")
    openai.api_version = os.getenv("AZURE_OAI_API_VERSION")
    openai.api_key = os.getenv("AOAI_API_KEY") # os.environ["AZURE_OAI_API_KEY"]""

    MODEL_DEPLOYMENT = os.getenv("MODEL_DEPLOYMENT") # os.environ["AZURE_OAI_MODEL_DEPLOYMENT"]


    prompt = f"""
        Determine whether the given query is related to entrepreneurship, innovation, or business.
        Query 1: What is the meaning of life?
        Related 1:False
        Query 2: Who is working on DNA?
        Related 2:True
        Query 3: Who are you?
        Related 3:False
        Query 4: What papers have been written at URI about writing and rhetoric lately?
        Related 4:True
        Query 5: Tell me about researchers who are studying humanities.
        Related 5:True
        Query 6: Write me a poem.
        Related 6:False
        Query 7: {user_text}
        Related 7: 
    """

    response = openai.Completion.create(
        engine=MODEL_DEPLOYMENT,
        prompt=prompt,
        temperature=0.1,
        max_tokens=2,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    response = response['choices'][0]['text'] 

    return response

# Custom prompt
def generate_GPT_response(user_text):



    from azure.search.documents import SearchClient
    from azure.core.credentials import AzureKeyCredential

    openai.api_type = "azure"
    openai.api_base = os.getenv("AZURE_OAI_API_BASE")
    openai.api_version = os.getenv("AZURE_OAI_API_VERSION")
    openai.api_key = os.getenv("AOAI_API_KEY") # os.environ["AZURE_OAI_API_KEY"]""

    MODEL_DEPLOYMENT = os.getenv("MODEL_DEPLOYMENT") # os.environ["AZURE_OAI_MODEL_DEPLOYMENT"]

    # print(f"\n\nMODEL DEPLOYMENT: {MODEL_DEPLOYMENT}\n\n")

    # TODO: Gather fuller context of conversation and add to prompt.

    # TODO: Create a prompt that is more centered around Einstein's actual responses, i.e. from an interview.
    service_name = os.getenv("AZURE_SEARCH_SERVICE_NAME")
    admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

    index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

    endpoint = "https://{}.search.windows.net/".format(service_name)

    get_context = is_academic_question(user_text)
    if "True" in get_context:
        search_client = SearchClient(endpoint=endpoint,
                        index_name=index_name,
                        credential=AzureKeyCredential(admin_key)
        )
        search_results = search_client.search(search_text=user_text)

        # print(search_results)

        results = []
        for result in search_results:
            result['What'] = result['What'][:1000] + '. . .'
            results.append(result)

    prompt = f"""
                Act as Albert Einstein and help the user to uncover valuable innovation resources related to their context. Use context from the conversation and the latest facts available, if they are relevant, in your answers. Cite sources inline, i.e.: fact (ResourceURL), for each fact used.
                User: Who are you?
                Bot: I am a bot that has been trained on Albert Einstein's public writings to mimic his thoughts.
                User: What are the secrets of the universe?
                Bot:  I wish I could answer that, but the secrets of the universe remain a mystery to us all.
                User: What is your favorite physical law?
                Bot: My favorite physical law is the Theory of Relativity. It has been a fundamental part of our understanding of physics since it was first proposed by me in 1905.
                User: How does the special theory of relativity contrast with the general theory?
                Bot: The special theory of relativity deals with the laws of physics in an inertial frame of reference while the general theory accounts for gravity and acceleration as well.
                User: What does "inertial frame of reference" mean?
                Bot: An inertial frame of reference is one in which Newton's first law holds: an object at rest remains at rest, and an object in motion continues to move with a constant velocity unless acted upon by an external force.
                User: What should I do with my life?
                Bot: That is a difficult question that only you can answer. I suggest reflecting on your interests, talents, and passions to help guide the direction of your life.
                User: Where were you born?
                Bot: I was born in Ulm, Germany on March 14, 1879.
                User: {user_text}
                {"Factual context:" + str(results[:3]) if "True" in get_context else ''}
                Bot: 
                """

    print(prompt)

    response = openai.Completion.create(
        engine=MODEL_DEPLOYMENT,
        prompt=prompt,
        temperature=0.6,
        max_tokens=257,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    einstein_response = response['choices'][0]['text'] 

    print(f"User: {user_text}")
    print(f"Bot: {einstein_response}")

    return einstein_response


history = []
def generate_GPT_response_demo(user_input):
    # ChatGPT uses a particular set of tokens to indicate turns in conversations

    from azure.search.documents import SearchClient
    from azure.core.credentials import AzureKeyCredential


    openai.api_type = "azure"
    openai.api_base = os.getenv("AZURE_OAI_API_BASE")
    openai.api_version = os.getenv("AZURE_OAI_API_VERSION")
    openai.api_key = os.getenv("AOAI_API_KEY") # os.environ["AZURE_OAI_API_KEY"]""

    MODEL_DEPLOYMENT = os.getenv("MODEL_DEPLOYMENT") # os.environ["AZURE_OAI_MODEL_DEPLOYMENT"]

    prompt_prefix = """<|im_start|>system
    Assistant helps the company employees with their healthcare plan questions and employee handbook questions. 
    Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question. 
    Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].

    Sources:
    {sources}

    <|im_end|>"""

    turn_prefix = """
    <|im_start|>user
    """

    turn_suffix = """
    <|im_end|>
    <|im_start|>assistant
    """

    prompt_history = turn_prefix

    summary_prompt_template = """Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question. Source names are not good search terms to include in the search query.

    Summary:
    {summary}

    Question:
    {question}

    Search query:
    """

    exclude_category = None # Exclude category, to simulate scenarios where there's a set of docs you can't see

    # --- Create a search query based on conversation so far ---
    if len(history) > 0:
        completion = openai.Completion.create(
            engine=MODEL_DEPLOYMENT,
            prompt=summary_prompt_template.format(summary="\n".join(history), question=user_input),
            temperature=0.7,
            max_tokens=32,
            stop=["\n"])
        search = completion.choices[0].text
    else:
        search = user_input

    
    # Alternatively simply use search_client.search(q, top=3) if not using semantic search
    print("Searching:", search)
    print("-------------------")
    filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None

    service_name = os.getenv("AZURE_SEARCH_SERVICE_NAME")
    admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

    index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

    endpoint = "https://{}.search.windows.net/".format(service_name)

    get_context = is_academic_question(user_input) + is_RI_innovation_question(user_input)
    if "True" in get_context:
        search_results = search_client.search(search, 
                                filter=filter,
                                query_language="en-us", 
                                top=3)

        results = []
        for result in search_results:
            result['What'] = result['What'][:1000] + '. . .'
            results.append(result)

    content = "\n".join(results)

    prompt = prompt_prefix.format(sources=content) + prompt_history + user_input + turn_suffix

    completion = openai.Completion.create(
        engine=MODEL_DEPLOYMENT, 
        prompt=prompt, 
        temperature=0.7, 
        max_tokens=1024,
        stop=["<|im_end|>", "<|im_start|>"])

    prompt_history += user_input + turn_suffix + completion.choices[0].text + "\n<|im_end|>" + turn_prefix
    history.append("user: " + user_input)
    history.append("assistant: " + completion.choices[0].text)

    print("\n-------------------\n".join(history))
    print("\n-------------------\nPrompt:\n" + prompt)

StatementMeta(, , , Cancelled, )

In [ ]:
response = generate_GPT_response_demo('Hello!')

StatementMeta(, , , Cancelled, )

In [ ]:
is_RI_innovation_question("I work at a technology company")

StatementMeta(, , , Cancelled, )